<a href="https://colab.research.google.com/github/zapatos24/luria_vote_data_closeout/blob/jeremy-wip/elaine_luria_vote_data_closeout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Elaine Luria Voter Data Closeout**

In [8]:
import pandas as pd
import requests
import json
import time

In [7]:
#Parse all counties associated with VA 02
house_district = '02'
counties = []
path = "https://results.elections.virginia.gov/vaelections/2020%20November%20General/Json/Member_House_of_Representatives_({}).json".format(house_district)

r = requests.get(path)

county_data = json.loads(r.text)
for locality in county_data['Localities']:
    counties.append(locality['Locality']['LocalityName'])

print(counties)

['ACCOMACK COUNTY', 'HAMPTON CITY', 'JAMES CITY COUNTY', 'NEWPORT NEWS CITY', 'NORFOLK CITY', 'NORTHAMPTON COUNTY', 'POQUOSON CITY', 'VIRGINIA BEACH CITY', 'WILLIAMSBURG CITY', 'YORK COUNTY']


In [17]:
#iterate through counties in district and pull data for each precinct into list of dictionaries
precinct_list_of_dict = []
luria_name = 'Elaine G. Luria'
taylor_name = 'Scott W. Taylor'
counties_no_space = [c.replace(' ', '_') for c in counties]

site_source = 'https://results.elections.virginia.gov/'
for county in counties_no_space:
    house_county_path = 'vaelections/2020%20November%20General/Json/Locality/{}/Member_House_of_Representatives_({}).json'.format(county, house_district)
    pres_county_path = 'vaelections/2020%20November%20General/Json/Locality/{}/President_and_Vice_President.json'.format(county)
    
    house_path = site_source + house_county_path
    pres_path = site_source + pres_county_path

    #iterate through house precinct data
    r = requests.get(house_path)
    precinct_data = json.loads(r.text)
    # print(precinct_data)
    # print(precinct_data['Precincts'])

    for p in precinct_data['Precincts']:
        p_dict = {}
        p_dict['Precinct'] = p['PrecinctName'][:-5]
        p_dict['State Senate District'] = ''
        p_dict['House District'] = ''
        p_dict['County/Locality'] = county

        for i in range(len(p['Candidates'])):
            if p['Candidates'][i]['BallotName'] == taylor_name:
                p_dict['Taylor Votes'] = p['Candidates'][i]['Votes']
                p_dict['Taylor Percentage'] = p['Candidates'][i]['Percentage']
            elif p['Candidates'][i]['BallotName'] == luria_name:
                p_dict['Luria Votes'] = p['Candidates'][i]['Votes']
                p_dict['Luria Percentage'] = p['Candidates'][i]['Percentage']

        precinct_list_of_dict.append(p_dict)
    time.sleep(.5)

for item in precinct_list_of_dict:
    print(item)     

{'Precinct': '# AB - Central Absentee Precinct', 'State Senate District': '', 'House District': '', 'County/Locality': 'ACCOMACK_COUNTY', 'Luria Votes': 5467, 'Luria Percentage': '63.64%', 'Taylor Votes': 2975, 'Taylor Percentage': '34.63%'}
{'Precinct': '## Provisional', 'State Senate District': '', 'House District': '', 'County/Locality': 'ACCOMACK_COUNTY', 'Luria Votes': 11, 'Luria Percentage': '50.00%', 'Taylor Votes': 11, 'Taylor Percentage': '50.00%'}
{'Precinct': '101 - CHINCOTEAGUE', 'State Senate District': '', 'House District': '', 'County/Locality': 'ACCOMACK_COUNTY', 'Luria Votes': 252, 'Luria Percentage': '19.30%', 'Taylor Votes': 1017, 'Taylor Percentage': '77.87%'}
{'Precinct': '201 - ATLANTIC', 'State Senate District': '', 'House District': '', 'County/Locality': 'ACCOMACK_COUNTY', 'Luria Votes': 98, 'Luria Percentage': '18.53%', 'Taylor Votes': 412, 'Taylor Percentage': '77.88%'}
{'Precinct': '202 - GREENBACKVILLE', 'State Senate District': '', 'House District': '', 'C

In [ ]:
#Convert list of dictionaries into pandas dataframe

In [ ]:
#

In [ ]:
export_cols = ["Precinct",
               "State Senate District",	
               "House District",
               "County/Locality",
               "Taylor Votes",
               "Taylor Percentage",
               "Luria Votes",
               "Luria Percentage",
               "Trump Votes",
               "Trump Percentage",
               "Biden Votes",
               "Biden Percentage"]